In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from src.data.sets import load_sets

X_train, y_train, X_val, y_val, X_test, y_test = load_sets()

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(911287, 10)
(911287,)
(303761, 10)
(303761,)
(303761, 10)
(303761,)


In [3]:
from src.models.pytorch import PytorchDataset

train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)

In [4]:
from src.models.pytorch import PytorchMultiClass

model = PytorchMultiClass(X_train.shape[1])

In [5]:
from src.models.pytorch import get_device

device = get_device()
model.to(device)

#change 10 to 6?

PytorchMultiClass(
  (layer_1): Linear(in_features=10, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=104, bias=True)
  (softmax): Softmax(dim=1)
)

In [6]:
print(model)

PytorchMultiClass(
  (layer_1): Linear(in_features=10, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=104, bias=True)
  (softmax): Softmax(dim=1)
)


In [7]:
criterion = nn.CrossEntropyLoss()

In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [9]:
N_EPOCHS = 50
BATCH_SIZE = 32

In [ ]:
from src.models.pytorch import train_classification, test_classification

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

Epoch: 0
	(train)	|	Loss: 0.1439	|	Acc: 5.7%
	(valid)	|	Loss: 0.1437	|	Acc: 6.3%
Epoch: 1
	(train)	|	Loss: 0.1439	|	Acc: 5.7%
	(valid)	|	Loss: 0.1437	|	Acc: 6.3%
